In [1]:
# Import necessary libraries
# pip install azure-search-documents==11.4.0b6
import openai
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI
from azure.core.credentials import AzureKeyCredential

In [2]:
# Configure the baseline configuration of the OpenAI library for Azure OpenAI Service & Azure Cognitive Search.
OPENAI_API_KEY = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"
OPENAI_API_BASE = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
OPENAI_DEPLOYMENT_NAME = "PLEASE_ENTER_YOUR_OWNED_AOAI_GPT_MODEL_NAME"
OPENAI_MODEL_NAME = "text-davinci-003"
OPENAI_EMBEDDING_DEPLOYMENT_NAME = "PLEASE_ENTER_YOUR_OWNED_AOAI_EMBEDDING_MODEL_NAME"
OPENAI_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
OPENAI_API_VERSION = "2023-05-15"
OPENAI_API_TYPE = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_version = OPENAI_API_VERSION
openai.api_type = OPENAI_API_TYPE
AZURE_COGNITIVE_SEARCH_ENDPOINT_NAME = "https://PLESAE_ENTER_YOUR_OWNED_ACS_RESOURCE_NAME.search.windows.net"
AZURE_COGNITIVE_SEARCH_INDEX_NAME = "PLEASE_ENTER_YOUR_OWNED_ACS_INDEX_NAME"
AZURE_COGNITIVE_SEARCH_KEY = "PLEASE_ENTER_YOUR_OWNED_ACS_SERVICE_KEY"
acs_credential = AzureKeyCredential(AZURE_COGNITIVE_SEARCH_KEY)

In [3]:
# Define the function to interact with Azure OpenAI Embedding Model.
embeddings = OpenAIEmbeddings(deployment=OPENAI_EMBEDDING_DEPLOYMENT_NAME, 
                                openai_api_key=OPENAI_API_KEY, 
                                model=OPENAI_EMBEDDING_MODEL_NAME, 
                                openai_api_type=OPENAI_API_TYPE, 
                                chunk_size=1)

In [4]:
# Define the function interact with Azure Cognitive Search. If the index does not exist, it will be created.
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=AZURE_COGNITIVE_SEARCH_ENDPOINT_NAME,
    azure_search_key=AZURE_COGNITIVE_SEARCH_KEY,
    index_name=AZURE_COGNITIVE_SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
)

In [5]:
# Define the query to search for the most similar documents.
query = "Who is the authors of this book?"
docs = vector_store.similarity_search(query)

In [6]:
# List the number of documents that are similar to the query.
len(docs)

4

In [8]:
# Print the most similar document.
print(docs[0].page_content)

58 Building a superhero team - Get your teammates to read this

Congratulations on finishing this book!

In Chapter 2, we talked about how this book can help you become the superhero of your team.

The only thing better than being a superhero is being part of a superhero team. I hope you’ll give copies of this book to your friends and teammates and help create other superheroes!

Page 118

Machine Learning Yearning-Draft

Andrew Ng


In [9]:
# Create a Question Answering chain using the embeddings and the similarity search.
chain = load_qa_chain(AzureOpenAI(openai_api_key=OPENAI_API_KEY, 
                                  deployment_name=OPENAI_DEPLOYMENT_NAME, 
                                  model_name=OPENAI_MODEL_NAME,
                                  openai_api_version=OPENAI_API_VERSION),
                                  chain_type="stuff")

In [10]:
# Perform first sample of question answering.
inquiry = "Who is the author of this book?"
docs = vector_store.similarity_search(inquiry)
chain.run(input_documents=docs, question=inquiry)

' The author of this book is Andrew Ng.'

In [11]:
# Perform second sample of question answering.
inquiry = "Please tell me the key summary of this book."
docs = vector_store.similarity_search(inquiry)
chain.run(input_documents=docs, question=inquiry)

' This book is about how to use machine learning to set technical direction for a machine learning project. It provides advice on how to read the clues that machine learning problems leave to save development time. It also provides advice on how to help team members understand why a particular direction is being recommended.'